In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, models

from CNNimagegen import create_image_updown_vol
from CNNimagegen import create_image_side_vol
from CNNimagegen import create_image_updown_price
from CNNimagegen import create_image_updown_vol_mid
from CNNimagegen import create_image_updown_vol_mid_triple_l10
from CNNimagegen import create_image_updown_vol_mid_triple_l5
from CNNimagegen import create_image_updown_vol_mid_triple_l3
from CNNimagegen import data_frame_organise

In [37]:
df = pd.read_csv(r"Stocks\MSFT\cleaned_data\MSFT_2019-01-03_34200000_57600000_orderbook_10.csv")
df = data_frame_organise(df)


c:\Users\dbjac\Desktop\CNN\CNNimagegen.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')  # Forward fill
c:\Users\dbjac\Desktop\CNN\CNNimagegen.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.

In [38]:

def create_cnn_2d_model_triple_channel(input_shape):
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Separate channels for Bid, Ask, and Mid
    green_channel = layers.Lambda(lambda x: x[:, :, :, 1:2])(inputs)  # Mid-price (Green, channel 1)
    red_channel = layers.Lambda(lambda x: x[:, :, :, 0:1])(inputs)    # Ask (Red, channel 0)
    blue_channel = layers.Lambda(lambda x: x[:, :, :, 2:3])(inputs)   # Bid (Blue, channel 2)

    # Convolution layers for each channel
    # Green (Mid) channel - 5x5 kernel
    green = layers.Conv2D(32, (5, 5), activation='relu', padding='same')(green_channel)
    green = layers.MaxPooling2D((2, 1))(green)
    green = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(green)
    green = layers.MaxPooling2D((2, 1))(green)

    # Red (Ask) channel - 3x3 kernel
    red = layers.Conv2D(32, (5, 5), activation='relu', padding='same')(red_channel)
    red = layers.MaxPooling2D((2, 1))(red)
    red = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(red)
    red = layers.MaxPooling2D((2, 1))(red)

    # Blue (Bid) channel - 3x3 kernel
    blue = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(blue_channel)
    blue = layers.MaxPooling2D((2, 1))(blue)
    blue = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(blue)
    blue = layers.MaxPooling2D((2, 1))(blue)

    # Concatenate outputs from all channels
    concatenated = layers.Concatenate()([green, red, blue])

    # Fully connected layers
    flat = layers.Flatten()(concatenated)
    dense = layers.Dense(128, activation='relu')(flat)
    outputs = layers.Dense(3, activation='softmax')(dense)  # 3 output classes (example)

    # Create model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model


In [39]:
"""def create_cnn_2d_model(input_shape):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (1, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((1, 2)),
        layers.Conv2D(64, (1, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((1, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    return model

"""

"def create_cnn_2d_model(input_shape):\n    model = keras.Sequential([\n        keras.Input(shape=input_shape),\n        layers.Conv2D(32, (1, 3), activation='relu', padding='same'),\n        layers.MaxPooling2D((1, 2)),\n        layers.Conv2D(64, (1, 3), activation='relu', padding='same'),\n        layers.MaxPooling2D((1, 2)),\n        layers.Flatten(),\n        layers.Dense(128, activation='relu'),\n        layers.Dense(3, activation='softmax')\n    ])\n    return model\n\n"

In [40]:
def create_cnn_2d_model_vertical(input_shape):
    num_features = input_shape[0] 
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (num_features, 1), activation='relu', padding='valid'),  
        layers.MaxPooling2D((1, 2)),
        layers.Conv2D(64, (num_features, 1), activation='relu', padding='valid'), 
        layers.MaxPooling2D((1, 2)),
        layers.Conv2D(128, (num_features, 1), activation='relu', padding='valid'), 
        layers.MaxPooling2D((1, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    return model

In [41]:
def create_target(df, prediction_horizon=10):
    target = np.zeros(len(df) - prediction_horizon)  # Initialize target array

    for i in range(len(df) - prediction_horizon):
      current_price = df['Mid'].iloc[i]
      future_price = df['Mid'].iloc[i + prediction_horizon]

      if future_price > current_price:
          target[i] = 2  # Up
      elif future_price < current_price:
          target[i] = 0  # Down
      else:
          target[i] = 1  # Neutral

    return target

In [42]:
def train_and_evaluate(df, grid_function, lookback, batch_size, epochs, prediction_horizon,num_channels):
    num_grids = len(df) - lookback
    
    grids = [grid_function(lookback + i, lookback, df) for i in range(num_grids)]
    grids_np = np.array(grids)
    print(grids_np.shape)

    print("Grids initialized")
    grids_reshaped = grids_np.reshape(-1, grids_np.shape[1], grids_np.shape[2], num_channels)
    
    # Targets
    targets = create_target(df,prediction_horizon) 

    # Train-test split
    train_size = int(len(grids_reshaped) * 0.8)  # 80% for training
    X_train, X_test = grids_reshaped[:train_size], grids_reshaped[train_size:]
    y_train, y_test = targets[:train_size], targets[train_size:]

    # Input shape
    input_shape = X_train.shape[1:]
    if num_channels == 1:
        model = create_cnn_2d_model_vertical(input_shape)  # Your CNN model creation function
    elif num_channels == 3:
        model = create_cnn_2d_model_triple_channel(input_shape)

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training Model")

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    print(f'Loss: {loss:.4f}')

    return history,model


In [43]:
def test_and_evaluate(df, grid_function, lookback, batch_size, epochs, num_channels):
    num_grids = len(df) - lookback
    #DEBUG
    og_grid = grid_function(lookback,lookback,df)
    # Targets
    sample_target = df["Target"].iloc[lookback] + 1
    print(sample_target)

    grids = [grid_function(lookback + i, lookback, df) for i in range(num_grids)]
    grids_np = np.array(grids)
    print(grids_np.shape)

    print("Grids initialized")
    grids_reshaped = grids_np.reshape(-1, grids_np.shape[1], grids_np.shape[2], num_channels)
    print(grids_reshaped.shape)
    re_grid = grids_reshaped[0]
    
    return og_grid , re_grid
    
    

    


In [47]:

lookback = 10
batch_size = 64
epochs = 10
prediction_horizon = 10

history_v5,model_5 = train_and_evaluate(df, create_image_updown_vol_mid_triple_l10, lookback,batch_size,epochs,prediction_horizon,3)
print("Model 5 complete")
#history_v6 = train_and_evaluate(df, create_image_updown_vol_mid_triple_l5, lookback,batch_size,epochs,3)
#print("Model 6 complete")
#history_v7,model_v7 = train_and_evaluate(df, create_image_updown_vol_mid_triple_l3, lookback,batch_size,epochs,3)
#print("Model 7 complete")

(197990, 50, 10, 3)
Grids initialized
Training Model


MemoryError: Unable to allocate 906. MiB for an array with shape (158392, 50, 10, 3) and data type float32

In [18]:
#Make predictions
predictions = model.predict(X_test_images)
predicted_labels = np.argmax(predictions, axis=1) #Axis 1 is for each sample
accuracy = accuracy_score(y_test_images.astype(int), predicted_labels)
print(f"Test Accuracy (using predictions): {accuracy}")

#Print classification report
print(classification_report(y_test_images.astype(int),predicted_labels))

#Print confusion matrix
print(confusion_matrix(y_test_images.astype(int),predicted_labels))

# Compare with model.evaluate (should be the same)
loss, accuracy_eval = model.evaluate(X_test_images, y_test_images, verbose=0)
print(f"Test Accuracy (using model.evaluate): {accuracy_eval}")

fig,ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(history.history["accuracy"],label="Train_Accuracy")
ax[0].plot(history.history["accuracy"],label="Val_Accuracy")
ax[1].plot(history.history["loss"],label="Train_Loss")
ax[1].plot(history.history["val_loss"],label="Val_Loss")
ax[0].legend()
ax[1].legend()


[<KerasVariable shape=(5, 5, 1, 32), dtype=float32, path=conv2d_18/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d_18/bias>,
 <KerasVariable shape=(5, 5, 1, 32), dtype=float32, path=conv2d_20/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d_20/bias>,
 <KerasVariable shape=(3, 3, 1, 32), dtype=float32, path=conv2d_22/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d_22/bias>,
 <KerasVariable shape=(5, 5, 32, 64), dtype=float32, path=conv2d_19/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=conv2d_19/bias>,
 <KerasVariable shape=(5, 5, 32, 64), dtype=float32, path=conv2d_21/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=conv2d_21/bias>,
 <KerasVariable shape=(3, 3, 32, 64), dtype=float32, path=conv2d_23/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=conv2d_23/bias>,
 <KerasVariable shape=(3456, 128), dtype=float32, path=dense_6/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=dense_6/bias>,
 <KerasV

In [ ]:
error
lookback = 10
batch_size = 32
epochs = 3


history_v1 = train_and_evaluate(df, create_image_updown_vol, lookback,batch_size,epochs,1)
print("Model 1 complete")
history_v2 = train_and_evaluate(df, create_image_side_vol, lookback,batch_size,epochs,1)
print("Model 2 complete")
history_v3 = train_and_evaluate(df, create_image_updown_price, lookback,batch_size,epochs,1)
print("Model 3 complete")
